In [1]:
import logging
from naslib.defaults.trainer import Trainer
from naslib.optimizers import DARTSOptimizer
from naslib.search_spaces import DartsSearchSpace
from naslib.utils import utils, setup_logger, get_config_from_args, set_seed, log_args
from naslib.search_spaces.core.graph import Graph, EdgeData
from naslib.search_spaces.core import primitives as ops
from torch import nn
from fvcore.common.config import CfgNode
from copy import deepcopy
from IPython.display import clear_output
import torch
from naslib.search_spaces.core.primitives import AbstractPrimitive

device: cuda:0
device: cpu
device: cuda:0
device: cuda:0
device: cuda:0
device: cuda:0


In [2]:
config = utils.get_config_from_args(config_type='nas')
config.optimizer = 'darts'
utils.set_seed(config.seed)
clear_output(wait=True)
utils.log_args(config)

logger = setup_logger(config.save + '/log.log')
logger.setLevel(logging.INFO)

Namespace(config_file='/project/dl2022s/robertsj/NASLib/naslib/benchmarks/nas_predictors/discrete_config.yaml', dist_backend='nccl', dist_url='tcp://127.0.0.1:8888', eval_only=False, gpu=None, model_path=None, multiprocessing_distributed=False, opts=[], rank=0, resume=False, seed=0, world_size=1)


In [3]:
class stack():
    def __init__(self):
        pass
    def __call__(self, tensors, edges_data=None):
        return torch.stack(tensors)


class SimpleSearchSpace(Graph):

    OPTIMIZER_SCOPE = [
        'a_stage_1',
        'a_stage_2'
    ]

    QUERYABLE = False

    def __init__(self):
        super().__init__()

        stages = ['a_stage_1', 'a_stage_2']

        # cell definition
        activation_cell = Graph()
        activation_cell.name = 'activation_cell'
        activation_cell.add_node(1) # input node
        activation_cell.add_node(2) # intermediate node
        activation_cell.add_node(3) # output node
        activation_cell.add_edges_from([(1, 2, EdgeData())]) # mutable intermediate edge
        activation_cell.add_edges_from([(2, 3, EdgeData().finalize())]) # immutable output edge

        # macroarchitecture definition
        self.name = 'makrograph'
        self.add_node(1) # input node
        self.add_node(2) # intermediate node
        for i, scope in zip(range(3, 5), stages):
            self.add_node(i, subgraph=deepcopy(activation_cell).set_scope(scope).set_input([i-1])) # activation cell i
            self.nodes[i]['subgraph'].name = scope
        self.add_node(5) # output node
        self.add_edges_from([(i, i+1, EdgeData()) for i in range(1, 5)])
        self.edges[1, 2].set('op',
            ops.Sequential(
                nn.Conv2d(3, 6, 5),
                nn.MaxPool2d(2),
                nn.Conv2d(6, 16, 5),
                nn.MaxPool2d(2),
                nn.Flatten()
            )) # convolutional edge
        self.edges[4, 5].set('op', 
            ops.Sequential(
                nn.Linear(400, 10), 
                nn.Softmax(dim=1)
            )) # linear edge
        
        for scope in stages:
            self.update_edges(
                update_func=lambda edge: self._set_ops(edge),
                scope=scope,
                private_edge_data=True,
            )

    def _set_ops(self, edge):
        edge.data.set('op', [
            ops.Sequential(nn.ReLU()),
            ops.Sequential(nn.Hardswish()),
            ops.Sequential(nn.LeakyReLU()),
            ops.Sequential(nn.Identity())
        ])

In [4]:
class Maximum(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        return torch.maximum(x[0],x[1])
    def get_embedded_ops(self):
        return None
    
class Minimum(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        return torch.minimum(x[0],x[1])
    def get_embedded_ops(self):
        return None


class stack():
    def __init__(self):
        pass
    def __call__(self, tensors, edges_data=None):
        return torch.stack(tensors)


class ComplexSearchSpace(Graph):

    OPTIMIZER_SCOPE = [
        'a_stage_1',
        'u_stage_1',
        'u_stage_2',
        'b_stage_1'
    ]

    QUERYABLE = False

    def __init__(self):
        super().__init__()

        u_stages = ['u_stage_1', 'u_stage_2']
        
        # unary cell definition
        unary_cell = Graph()
        unary_cell.name = 'u_cell'
        unary_cell.add_node(1) # input node
        unary_cell.add_node(2) # intermediate node
        unary_cell.add_node(3) # output node
        unary_cell.add_edges_from([(1, 2, EdgeData())]) # mutable edge
        unary_cell.edges[1, 2].set('cell_name', 'u_cell')
        unary_cell.add_edges_from([(2, 3, EdgeData().finalize())]) # immutable edge
        
        # binary cell definition
        binary_cell = Graph()
        binary_cell.name = 'b_cell'
        binary_cell.add_node(1) # input node
        binary_cell.add_node(2) # input node
        binary_cell.add_node(3) # concatination node
        binary_cell.nodes[3]['comb_op'] = stack()
        binary_cell.add_node(4) # intermediate node
        binary_cell.add_node(5) # output node
        binary_cell.add_edges_from([(3, 4, EdgeData())]) # mutable edge
        binary_cell.edges[3, 4].set('cell_name', 'b_cell') 
        binary_cell.add_edges_from([(1, 3, EdgeData().finalize()),
                                    (2, 3, EdgeData().finalize()),
                                    (4, 5, EdgeData().finalize())]) # immutable edges
        
        # activation cell definition
        activation_cell = Graph()
        activation_cell.name = 'a_cell'
        activation_cell.add_node(1) # input node
        activation_cell.add_node(2, subgraph=deepcopy(unary_cell).set_scope('u_stage_1').set_input([1])) # unary node
        activation_cell.nodes[2]['subgraph'].name = 'u_stage_1'
        activation_cell.add_node(3, subgraph=deepcopy(unary_cell).set_scope('u_stage_2').set_input([1])) # unary node
        activation_cell.nodes[3]['subgraph'].name = 'u_stage_2'
        activation_cell.add_node(4, subgraph=deepcopy(binary_cell).set_scope('b_stage_1').set_input([2, 3])) # binary node
        activation_cell.nodes[4]['subgraph'].name = 'b_stage_1'
        activation_cell.add_node(5) # output node
        activation_cell.add_edges_from([(1, 2, EdgeData().finalize()), 
                                        (1, 3, EdgeData().finalize()),
                                        (2, 4, EdgeData().finalize()),
                                        (3, 4, EdgeData().finalize()), 
                                        (4, 5, EdgeData().finalize())])
        
        # macroarchitecture definition
        self.name = 'makrograph'
        self.add_node(1) # input node
        self.add_node(2) # intermediate node
        self.add_node(3, subgraph=deepcopy(activation_cell).set_input([2])) # activation cell
        self.nodes[3]['subgraph'].name = 'a_stage_1'
        self.add_node(4) # output node
        self.add_edges_from([(i, i+1, EdgeData()) for i in range(1, 4)])
        self.edges[1, 2].set('op',
            ops.Sequential(
                nn.Conv2d(3, 6, 5),
                nn.MaxPool2d(2),
                nn.Conv2d(6, 16, 5),
                nn.MaxPool2d(2),
                nn.Flatten()
            )) # convolutional edge
        self.edges[3, 4].set('op', 
            ops.Sequential(
                nn.Linear(400, 10), 
                nn.Softmax(dim=1)
            )) # linear edge
        
        for scope in u_stages:
            self.update_edges(
                update_func=lambda edge: self._set_unary_ops(edge),
                scope=scope,
                private_edge_data=True,
            ) # set unary cell ops
        
        self.update_edges(
            update_func=lambda edge: self._set_binary_ops(edge),
            scope='b_stage_1',
            private_edge_data=True
        ) # set binary cell ops
        

    def _set_unary_ops(self, edge):
        edge.data.set('op', [ops.Identity(), ops.Zero(stride=1)]) 
        
        
    def _set_binary_ops(self, edge):
        edge.data.set('op', [Minimum(), Maximum()]) 

In [27]:
class Power(AbstractPrimitive):
    def __init__(self,power):
        super().__init__(locals())
        self.power=power
    def forward(self,x, edge_data=None):
        return torch.pow(x,self.power)
    def get_embedded_ops(self):
        return None

class Sin(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        return torch.sin(x)
    def get_embedded_ops(self):
        return None
    
class Cos(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        return torch.cos(x)
    def get_embedded_ops(self):
        return None

class Abs_op(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        return torch.abs(x)
    def get_embedded_ops(self):
        return None

class Sign(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        return x*-1
    def get_embedded_ops(self):
        return None

class Beta_mul(AbstractPrimitive):
    def __init__(self, channels):
        super().__init__(locals())
        self.beta = torch.nn.Parameter(torch.ones(channels))
    def forward(self,x, edge_data=None):
        return x * self.beta
    def get_embedded_ops(self):
        return None

class Beta_add(AbstractPrimitive):
    def __init__(self, channels):
        super().__init__(locals())
        self.beta = torch.nn.Parameter(torch.ones(channels))
    def forward(self,x, edge_data=None):
        return x + self.beta
    def get_embedded_ops(self):
        return None

class Log(AbstractPrimitive):
    def __init__(self,eps=1e-10):
        super().__init__(locals())
        self.eps = eps
    def forward(self,x, edge_data=None):
        return torch.log(x+self.eps)
    def get_embedded_ops(self):
        return None

class Exp(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        return torch.exp(x)
    def get_embedded_ops(self):
        return None

class Sinh(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        return torch.sinh(x)
    def get_embedded_ops(self):
        return None

class Cosh(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        return torch.cosh(x)
    def get_embedded_ops(self):
        return None

class Tanh(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        return torch.tanh(x)
    def get_embedded_ops(self):
        return None

class Asinh(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        return torch.asinh(x)
    def get_embedded_ops(self):
        return None

class Acosh(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        return torch.acosh(x)
    def get_embedded_ops(self):
        return None
    
class Atan(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        return torch.atan(x)
    def get_embedded_ops(self):
        return None

class Sinc(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        return torch.sinc(x)
    def get_embedded_ops(self):
        return None

class Maximum0(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        return torch.maximum(x,torch.zeros(x.shape).cuda())
    def get_embedded_ops(self):
        return None
    
class Minimum0(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        return torch.minimum(x,torch.zeros(x.shape).cuda())
    def get_embedded_ops(self):
        return None
    
class Sigmoid(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        return torch.sigmoid(x)
    def get_embedded_ops(self):
        return None

class LogExp(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        return torch.log(1+torch.exp(x))
    def get_embedded_ops(self):
        return None

class Exp2(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        return torch.exp(-torch.pow(x,2))
    def get_embedded_ops(self):
        return None

class Erf(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        return torch.erf(x)
    def get_embedded_ops(self):
        return None

class Beta(AbstractPrimitive):
    def __init__(self, channels):
        super().__init__(locals())
        self.beta = torch.nn.Parameter(torch.ones(channels))
    def forward(self,x, edge_data=None):
        return self.beta
    def get_embedded_ops(self):
        return None
    
class Add(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        return torch.add(x[0],x[1])
    def get_embedded_ops(self):
        return None

class Sub(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        return torch.sub(x[0],x[1])
    def get_embedded_ops(self):
        return None

class Mul(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        return torch.mul(x[0],x[1])
    def get_embedded_ops(self):
        return None

class Div(AbstractPrimitive):
    def __init__(self,eps=1e-10):
        super().__init__(locals())
        self.eps=eps
    def forward(self,x, edge_data=None):
        return torch.div(x[0],x[1] + self.eps)
    def get_embedded_ops(self):
        return None

class Maximum(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        return torch.maximum(x[0],x[1])
    def get_embedded_ops(self):
        return None
    
class Minimum(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        return torch.minimum(x[0],x[1])
    def get_embedded_ops(self):
        return None

class SigMul(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        return torch.mul(torch.sigmoid(x[0]),x[1])
    def get_embedded_ops(self):
        return None

class ExpBetaSub2(AbstractPrimitive):
    def __init__(self, channels):
        super().__init__(locals())
        self.beta = torch.nn.Parameter(torch.ones(channels))
    def forward(self,x, edge_data=None):
        return torch.exp(-self.beta*torch.pow(torch.sub(x[0],x[1]),2))
    def get_embedded_ops(self):
        return None

class ExpBetaSubAbs(AbstractPrimitive):
    def __init__(self, channels):
        super().__init__(locals())
        self.beta = torch.nn.Parameter(torch.ones(channels))
    def forward(self,x, edge_data=None):
        return torch.exp(-self.beta*torch.abs(torch.sub(x[0],x[1])))
    def get_embedded_ops(self):
        return None

class BetaMix(AbstractPrimitive):
    def __init__(self, channels):
        super().__init__(locals())
        self.beta = torch.nn.Parameter(torch.ones(channels))
    def forward(self,x, edge_data=None):
        return torch.add(-self.beta*x[0],(1-self.beta)*x[1])
    def get_embedded_ops(self):
        return None

In [141]:
class RNNSearchSpace(Graph):

    OPTIMIZER_SCOPE = [
        'a_stage_1',
        'u_stage_1',
        'u_stage_2',
        'u_stage_3',
        'u_stage_4',
        'b_stage_1',
        'b_stage_2'
    ]

    QUERYABLE = False

    def __init__(self):
        super().__init__()

        u_stages = ['u_stage_1', 'u_stage_2', 'u_stage_3', 'u_stage_4']
        b_stages = ['b_stage_1', 'b_stage_2']
        
        # unary cell definition
        unary_cell = Graph()
        unary_cell.name = 'u_cell'
        unary_cell.add_node(1) # input node
        unary_cell.add_node(2) # intermediate node
        unary_cell.add_node(3) # output node
        unary_cell.add_edges_from([(1, 2, EdgeData())]) # mutable edge
        unary_cell.edges[1, 2].set('cell_name', 'u_cell')
        unary_cell.add_edges_from([(2, 3, EdgeData().finalize())]) # immutable edge
        
        # binary cell definition
        binary_cell = Graph()
        binary_cell.name = 'b_cell'
        binary_cell.add_node(1) # input node
        binary_cell.add_node(2) # input node
        binary_cell.add_node(3) # concatination node
        binary_cell.nodes[3]['comb_op'] = stack()
        binary_cell.add_node(4) # intermediate node
        binary_cell.add_node(5) # output node
        binary_cell.add_edges_from([(3, 4, EdgeData())]) # mutable edge
        binary_cell.edges[3, 4].set('cell_name', 'b_cell') 
        binary_cell.add_edges_from([(1, 3, EdgeData().finalize()),
                                    (2, 3, EdgeData().finalize()),
                                    (4, 5, EdgeData().finalize())]) # immutable edges
        
        # activation cell definition
        activation_cell = Graph()
        activation_cell.name = 'a_cell'
        activation_cell.add_node(1) # input node
        activation_cell.add_node(2, subgraph=deepcopy(unary_cell).set_scope('u_stage_1').set_input([1])) # unary cell 1
        activation_cell.nodes[2]['subgraph'].name = 'u_stage_1'
        activation_cell.add_node(3, subgraph=deepcopy(unary_cell).set_scope('u_stage_2').set_input([1])) # unary cell 2
        activation_cell.nodes[3]['subgraph'].name = 'u_stage_2'
        activation_cell.add_node(4, subgraph=deepcopy(unary_cell).set_scope('u_stage_3').set_input([1])) # unary cell 3
        activation_cell.nodes[4]['subgraph'].name = 'u_stage_3'
        activation_cell.add_node(5, subgraph=deepcopy(binary_cell).set_scope('b_stage_1').set_input([2, 3])) # binary cell 1
        activation_cell.nodes[5]['subgraph'].name = 'b_stage_1'
        activation_cell.add_node(6, subgraph=deepcopy(unary_cell).set_scope('u_stage_4').set_input([5])) # unary cell 4
        activation_cell.nodes[6]['subgraph'].name = 'u_stage_4'
        activation_cell.add_node(7, subgraph=deepcopy(binary_cell).set_scope('b_stage_2').set_input([4, 6])) # binary cell 2
        activation_cell.nodes[7]['subgraph'].name = 'b_stage_2'
        activation_cell.add_node(8) # output node
        activation_cell.add_edges_from([(1, 2, EdgeData().finalize()), 
                                        (1, 3, EdgeData().finalize()),
                                        (1, 4, EdgeData().finalize()),
                                        (2, 5, EdgeData().finalize()),
                                        (3, 5, EdgeData().finalize()), 
                                        (4, 7, EdgeData().finalize()),
                                        (5, 6, EdgeData().finalize()),
                                        (6, 7, EdgeData().finalize()),
                                        (7, 8, EdgeData().finalize())])
        
        # macroarchitecture definition
        self.name = 'makrograph'
        self.add_node(1) # input node
        self.add_node(2) # intermediate node
        self.add_node(3, subgraph=deepcopy(activation_cell).set_input([2])) # activation cell
        self.nodes[3]['subgraph'].name = 'a_stage_1'
        self.add_node(4) # output node
        self.add_edges_from([(i, i+1, EdgeData()) for i in range(1, 4)])
        self.edges[1, 2].set('op',
            ops.Sequential(
                nn.Conv2d(3, 6, 5),
                nn.MaxPool2d(2),
                nn.Conv2d(6, 16, 5),
                nn.MaxPool2d(2),
                nn.Flatten()
            )) # convolutional edge
        self.edges[3, 4].set('op', 
            ops.Sequential(
                nn.Linear(400, 10), 
                nn.Softmax(dim=1)
            )) # linear edge
        
        for scope in u_stages:
            self.update_edges(
                update_func=lambda edge: self._set_unary_ops(edge),
                scope=scope,
                private_edge_data=True,
            ) # set unary cell ops
        
        for scope in b_stages:
            self.update_edges(
                update_func=lambda edge: self._set_binary_ops(edge),
                scope=scope,
                private_edge_data=True
            ) # set binary cell ops
        

    def _set_unary_ops(self, edge, channels=None):
        edge.data.set('op', [
            ops.Identity(), 
            ops.Zero(stride=1),
            Sign(),
            Abs_op(),
            Power(2),
            Power(3),
#             Power(0.5),
#             Beta_mul(channels),
#             Beta_add(channels),
#             Log(),
            Exp(),
            Sin(),
            Cos(),
            Sinh(),
            Cosh(),
            Tanh(),
            Asinh(),
#             Acosh(),
            Atan(),
            Sinc(),
            Maximum0(),
            Minimum0(),
            Sigmoid(),
            LogExp(),
            Exp2(),
            Erf(),
#             Beta(channels)
        ]) 
        
        
    def _set_binary_ops(self, edge, channels=None):
        edge.data.set('op', [
            Add(),
            Sub(),
            Mul(),
            Div(),
            Minimum(),
            Maximum(),
            SigMul(),
#             ExpBetaSub2(channels),
#             ExpBetaSubAbs(channels),
#             BetaMix(channels)
        ]) 

In [142]:
search_space = RNNSearchSpace()

In [143]:
optimizer = DARTSOptimizer(config)
optimizer.adapt_search_space(search_space)

[07/04 14:25:37 nl.optimizers.oneshot.darts.optimizer]: Parsed graph:
Graph a_stage_1:
 Graph(
  (a_stage_1-edge(1,2)): Identity()
  (a_stage_1-edge(1,3)): Identity()
  (a_stage_1-edge(1,4)): Identity()
  (a_stage_1-subgraph_at(2)): Graph u_stage_1-0.0467797, scope u_stage_1, 3 nodes
  (a_stage_1-edge(2,5)): Identity()
  (a_stage_1-subgraph_at(3)): Graph u_stage_2-0.0467797, scope u_stage_2, 3 nodes
  (a_stage_1-edge(3,5)): Identity()
  (a_stage_1-subgraph_at(4)): Graph u_stage_3-0.0467797, scope u_stage_3, 3 nodes
  (a_stage_1-edge(4,7)): Identity()
  (a_stage_1-subgraph_at(5)): Graph b_stage_1-0.4453522, scope b_stage_1, 5 nodes
  (a_stage_1-edge(5,6)): Identity()
  (a_stage_1-subgraph_at(6)): Graph u_stage_4-0.0467797, scope u_stage_4, 3 nodes
  (a_stage_1-edge(6,7)): Identity()
  (a_stage_1-subgraph_at(7)): Graph b_stage_2-0.4453522, scope b_stage_2, 5 nodes
  (a_stage_1-edge(7,8)): Identity()
)
Graph b_stage_1:
 Graph(
  (b_stage_1-edge(1,3)): Identity()
  (b_stage_1-edge(2,3)): I

In [144]:
trainer = Trainer(optimizer, config)
trainer.search()

[07/04 14:25:40 nl.defaults.trainer]: param size = 0.006882MB
[07/04 14:25:40 nl.defaults.trainer]: Start training
Files already downloaded and verified
Files already downloaded and verified
[07/04 14:25:41 nl.optimizers.oneshot.darts.optimizer]: Arch weights (alphas, last column argmax): 
+0.001590, -0.000726, -0.001156, +0.001044, -0.000165, +0.000628, -0.000305, 0
-0.000143, +0.000711, -0.000879, -0.000593, +0.000674, +0.000690, -0.000997, 1
+0.000266, -0.000916, +0.000696, +0.000741, +0.001266, -0.001623, +0.000183, -0.000675, +0.000176, -0.000330, +0.000302, +0.000080, +0.000234, -0.001364, -0.000962, +0.000270, +0.000273, -0.000181, -0.000370, -0.001080, -0.000125, 4
+0.002006, -0.000984, -0.000772, -0.000796, +0.001256, -0.000275, -0.000914, +0.001006, -0.001760, -0.000923, +0.000914, -0.000407, +0.000984, -0.000541, +0.000173, -0.001080, -0.000163, +0.000123, -0.000198, +0.000291, -0.000324, 0
-0.000489, -0.000153, -0.000786, +0.000496, -0.000106, +0.001298, +0.001919, +0.00169

KeyboardInterrupt: 

In [ ]:
trainer.evaluate_oneshot()